In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1644,application_1765289937462_1629,pyspark,idle,Link,Link,None,
1667,application_1765289937462_1651,pyspark,idle,Link,Link,None,
1673,application_1765289937462_1657,pyspark,idle,Link,Link,None,
1680,application_1765289937462_1664,pyspark,idle,Link,Link,None,
1681,application_1765289937462_1665,pyspark,idle,Link,Link,None,
1682,application_1765289937462_1666,pyspark,idle,Link,Link,None,
1686,application_1765289937462_1670,pyspark,idle,Link,Link,None,
1687,application_1765289937462_1671,pyspark,idle,Link,Link,None,
1688,application_1765289937462_1672,pyspark,idle,Link,Link,None,
1691,application_1765289937462_1675,pyspark,busy,Link,Link,None,


In [2]:
import time

from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql.functions import year, to_timestamp, col, desc, count, sum, round, row_number, substring
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1693,application_1765289937462_1677,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
CRIMES_PATH_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv"
CRIMES_PATH_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv"
RE_PATH = "s3://initial-notebook-data-bucket-dblab-905418150721/project_data/RE_codes.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName("Query 2").getOrCreate()

def load_crime_subset(path):
    df = spark.read.option("header", "true").csv(path)

    return df.select(
        col("DATE OCC"), 
        col("Vict Descent")
    )

crime1 = load_crime_subset(CRIMES_PATH_1)
crime2 = load_crime_subset(CRIMES_PATH_2)
crimes = crime1.union(crime2)

re_codes = spark.read.option("header", "true").csv(RE_PATH)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
crimes.show(n=10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|            DATE OCC|Vict Descent|
+--------------------+------------+
|2010 Feb 20 12:00...|           H|
|2010 Sep 12 12:00...|           W|
|2010 Aug 09 12:00...|           H|
|2010 Jan 05 12:00...|           W|
|2010 Jan 02 12:00...|           H|
|2010 Jan 04 12:00...|           B|
|2010 Jan 07 12:00...|           H|
|2010 Jan 08 12:00...|           B|
|2010 Jan 09 12:00...|           H|
|2010 Jan 06 12:00...|           W|
+--------------------+------------+
only showing top 10 rows

In [16]:
re_codes.show(n=10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|Vict Descent|   Vict Descent Full|
+------------+--------------------+
|           A|         Other Asian|
|           B|               Black|
|           C|             Chinese|
|           D|           Cambodian|
|           F|            Filipino|
|           G|           Guamanian|
|           H|Hispanic/Latin/Me...|
|           I|American Indian/A...|
|           J|            Japanese|
|           K|              Korean|
+------------+--------------------+
only showing top 10 rows

In [5]:
df_api_start = time.time()

formatted_crimes = crimes.withColumn(
    "year", 
    year(to_timestamp(col("DATE OCC"), "yyyy MMM dd hh:mm:ss a"))
)

joined_df = formatted_crimes.join(re_codes, "Vict Descent")

grouped_df = joined_df.groupBy("year", "Vict Descent Full") \
    .agg(count("*").alias("victims_count"))

window_year = Window.partitionBy("year")
window_rank = Window.partitionBy("year").orderBy(col("victims_count").desc())

final_df = grouped_df \
    .withColumn("total_year", sum("victims_count").over(window_year)) \
    .withColumn("percentage", round((col("victims_count") / col("total_year")) * 100, 1)) \
    .withColumn("rank", row_number().over(window_rank)) \
    .filter(col("rank") <= 3) \
    .select(
        col("year"),
        col("Vict Descent Full").alias("Victim Descent"),
        col("victims_count").alias("#"),
        col("percentage").alias("%")
    ) \
    .orderBy(desc("year"), desc("#"))

final_df.show(truncate=False)

df_api_end = time.time()
df_time = df_api_end - df_api_start

print(f"DataFrame Time: {df_time} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+----+
|year|Victim Descent        |#    |%   |
+----+----------------------+-----+----+
|2025|Hispanic/Latin/Mexican|34   |40.5|
|2025|Unknown               |24   |28.6|
|2025|White                 |13   |15.5|
|2024|Hispanic/Latin/Mexican|28576|29.1|
|2024|White                 |22958|23.3|
|2024|Unknown               |19984|20.3|
|2023|Hispanic/Latin/Mexican|69401|34.6|
|2023|White                 |44615|22.2|
|2023|Black                 |30504|15.2|
|2022|Hispanic/Latin/Mexican|73111|35.6|
|2022|White                 |46695|22.8|
|2022|Black                 |34634|16.9|
|2021|Hispanic/Latin/Mexican|63676|35.1|
|2021|White                 |44523|24.5|
|2021|Black                 |30173|16.6|
|2020|Hispanic/Latin/Mexican|61606|35.3|
|2020|White                 |42638|24.5|
|2020|Black                 |28785|16.5|
|2019|Hispanic/Latin/Mexican|72458|36.4|
|2019|White                 |48863|24.5|
+----+----------------------+-----+----+
only showing top

In [5]:
crime1.union(crime2).createOrReplaceTempView("crime_table")
re_codes.createOrReplaceTempView("codes_table")

sql_start = time.time()

query = """
WITH 
cleaned_data AS (
    SELECT 
        year(to_timestamp(`DATE OCC`, 'yyyy MMM dd hh:mm:ss a')) as crime_year,
        `Vict Descent`
    FROM crime_table
    WHERE `DATE OCC` IS NOT NULL
),

-- join crime x codes   
joined_data AS (
    SELECT 
        c.crime_year,
        r.`Vict Descent Full` 
    FROM cleaned_data c
    JOIN codes_table r ON c.`Vict Descent` = r.`Vict Descent` 
),

-- count crimes per year
counts_per_year AS (
    SELECT 
        crime_year,
        `Vict Descent Full`,
        COUNT(*) as victims_count
    FROM joined_data
    GROUP BY crime_year, `Vict Descent Full`
),

-- rank & total sum
windowed_calcs AS (
    SELECT 
        crime_year,
        `Vict Descent Full`,
        victims_count,
        -- ypologismos sum ana year gia pososto
        SUM(victims_count) OVER (PARTITION BY crime_year) as total_year,
        -- rank
        ROW_NUMBER() OVER (PARTITION BY crime_year ORDER BY victims_count DESC) as rnk
    FROM counts_per_year
)

-- top 3
SELECT 
    crime_year as `year`,
    `Vict Descent Full` as `Victim Descent`,
    victims_count as `#`,
    -- pososto
    ROUND((victims_count / total_year) * 100, 1) as `%`
FROM windowed_calcs
WHERE rnk <= 3
ORDER BY `year` DESC, `#` DESC
"""

# Εκτέλεση του SQL Query
result_sql = spark.sql(query)
result_sql.show(truncate=False)

sql_end = time.time()
print(f"SQL API Time: {sql_end - sql_start:.4f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+----+
|year|Victim Descent        |#    |%   |
+----+----------------------+-----+----+
|2025|Hispanic/Latin/Mexican|34   |40.5|
|2025|Unknown               |24   |28.6|
|2025|White                 |13   |15.5|
|2024|Hispanic/Latin/Mexican|28576|29.1|
|2024|White                 |22958|23.3|
|2024|Unknown               |19984|20.3|
|2023|Hispanic/Latin/Mexican|69401|34.6|
|2023|White                 |44615|22.2|
|2023|Black                 |30504|15.2|
|2022|Hispanic/Latin/Mexican|73111|35.6|
|2022|White                 |46695|22.8|
|2022|Black                 |34634|16.9|
|2021|Hispanic/Latin/Mexican|63676|35.1|
|2021|White                 |44523|24.5|
|2021|Black                 |30173|16.6|
|2020|Hispanic/Latin/Mexican|61606|35.3|
|2020|White                 |42638|24.5|
|2020|Black                 |28785|16.5|
|2019|Hispanic/Latin/Mexican|72458|36.4|
|2019|White                 |48863|24.5|
+----+----------------------+-----+----+
only showing top

| Method | Execution Time (s) |
| :--- | :--- |
| DataFrame API | 17.944045782089233 |
| SQL API | 16.4828 |